In [2]:
import os
import sys
import pandas
import numpy, scipy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [141]:
from  pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [61]:
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [3]:
from pyspark.ml.regression import GeneralizedLinearRegression, LinearRegression
from pyspark.ml.classification import LogisticRegression

In [4]:
file_name = "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/dat_test.tsv"

In [5]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [6]:
spark.version

'2.3.0'

In [175]:
df = spark.read.csv(file_name, sep="\t", header=True)

In [176]:
for i, x in enumerate(df.columns):
    if x.startswith("cells") or x.startswith("is_infected"):
        df = df.withColumn(x, df[x].cast("double"))
df = df.fillna(0)

In [177]:
cols = [x for x in df.columns if x.startswith("cell")]

In [178]:
assembler = VectorAssembler(
          inputCols=cols,
          outputCol="features")
df = assembler.transform(df)

In [179]:
glr = GeneralizedLinearRegression(maxIter=100, regParam=1, labelCol="is_infected", family="binomial", link="logit")

In [180]:
model = glr.fit(df)

In [198]:
beta = scipy.append(scipy.array(model.intercept),scipy.array(model.coefficients))

In [204]:
ts = scipy.zeros_like(beta) * scipy.NaN
ts

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])

In [197]:
try:
    print(model.summary.)
except Exception as e:
    print("e")

e


In [151]:
model.transform(test).select(["is_infected","prediction", "rawPrediction", "probability"]).take(5)

AnalysisException: "cannot resolve '`rawPrediction`' given input columns: [replicate, object_idx, is_infected, cells_areashape_formfactor, nuclei_children_perinuclei_count, library, cells_neighbors_secondclosestxvector_2, perinuclei_areashape_minoraxislength, nuclei_areashape_orientation, nuclei_location_center_y, cells_parent_nuclei, plate, perinuclei_areashape_orientation, perinuclei_parent_expandednuclei, design, nuclei_areashape_majoraxislength, perinuclei_areashape_majoraxislength, perinuclei_parent_nuclei, nuclei_areashape_formfactor, study, nuclei_areashape_minoraxislength, nuclei_children_cells_count, gene, prediction, cells_neighbors_firstclosestobjectnumber_2, cells_neighbors_anglebetweenneighbors_2, perinuclei_location_center_y, perinuclei_areashape_formfactor, nuclei_location_center_x, nuclei_areashape_area, cells_areashape_majoraxislength, perinuclei_areashape_perimeter, features, cells_neighbors_secondclosestobjectnumber_2, cells_location_center_x, sirna, cells_areashape_

In [41]:
beta = scipy.append(scipy.array(model.intercept), scipy.array(model.coefficients))
beta

array([ -40.58522438,  -18.09379892,   49.98335339,   -8.53411228,
          5.78628766,  -91.76619448,  139.77918   ,   11.78750353,
         47.38566867,    0.        , -155.22540079,  -47.64358771,
          0.61939133,   20.95393594,  -30.7513961 ,   -3.94608597,
          7.78275043,   38.46303947,  -41.13395916,  -22.45524179,
         76.49301044])

In [42]:
pred = model.transform(test)

In [95]:
s = model.summary

In [89]:
s.predictions.select("prediction", "rawPrediction", "probability").take(10)

[Row(prediction=1.0, rawPrediction=DenseVector([-19.7205, 19.7205]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=1.0, rawPrediction=DenseVector([-720.2587, 720.2587]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=0.0, rawPrediction=DenseVector([348.458, -348.458]), probability=DenseVector([1.0, 0.0])),
 Row(prediction=0.0, rawPrediction=DenseVector([14.9133, -14.9133]), probability=DenseVector([1.0, 0.0])),
 Row(prediction=1.0, rawPrediction=DenseVector([-2148.484, 2148.484]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=1.0, rawPrediction=DenseVector([-47.7827, 47.7827]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=1.0, rawPrediction=DenseVector([-83.4945, 83.4945]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=0.0, rawPrediction=DenseVector([66.3773, -66.3773]), probability=DenseVector([1.0, 0.0])),
 Row(prediction=1.0, rawPrediction=DenseVector([-456.6449, 456.6449]), probability=DenseVector([0.0, 1.0])),
 Row(prediction=1.0, rawPredict

In [133]:
pred = s.predictions.select("probability")
pred = pred.rdd.map(lambda x: x[0][1])

In [135]:
pred = pred.map(lambda x: x * (1 - x))

[0.9999999972742135,
 1.0,
 4.641162548908174e-152,
 3.3362159533567064e-07,
 1.0,
 1.0,
 1.0,
 1.4883962383560773e-29,
 1.0,
 0.9999999999307396]

In [85]:
s.preadictions.select("prediction").toPandas().values[:,0]

array([1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1.])

In [63]:
preds = RowMatrix(s.predictions.select("prediction").rdd)

In [73]:
w = preds.rows.map(lambda x: x * (1 -  x))

In [79]:
w.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 54.0 failed 1 times, most recent failure: Lost task 0.0 in stage 54.0 (TID 54, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/spark/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 83, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))
TypeError: Cannot convert

In [156]:
dataset = spark.read.format("libsvm").load("/usr/local/spark/spark/data/mllib/sample_libsvm_data.txt")

In [159]:
dataset.take(5)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

In [163]:
dmodel = pyspark.ml.regression.GeneralizedLinearRegression(
    maxIter=100, family="binomial", link="logit", regParam=0.0).fit(dataset)

In [164]:
dmodel.summary

Py4JJavaError: An error occurred while calling o2079.toString.
: java.lang.UnsupportedOperationException: No summary available for this GeneralizedLinearRegressionModel
	at org.apache.spark.ml.regression.GeneralizedLinearRegressionTrainingSummary.toString(GeneralizedLinearRegression.scala:1566)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:497)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)


In [165]:
import pandas as pd

In [166]:
df = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [167]:
df[['admit']] = df[['admit']].astype(float)

In [168]:
df = spark.createDataFrame(df)

In [169]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# indexer for categorical features
rank_indexer = StringIndexer(inputCol = 'rank', outputCol = 'rank_ind', handleInvalid="skip")
# encoder for categorical features
rank_encoder = OneHotEncoder(inputCol = 'rank_ind', outputCol = 'rank_enc')

assembler = VectorAssembler(inputCols=['gre','gpa','rank_enc'], outputCol="features")

from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="binomial", link = 'logit', maxIter=10, labelCol="admit", featuresCol="features")

stages = [rank_indexer] + [rank_encoder] + [assembler] + [glr]

from pyspark.ml import Pipeline
final_pipeline = Pipeline(
    stages = stages
)

In [170]:
pipelineModel = final_pipeline.fit(df)
glr = pipelineModel.stages[3]
transformed_data = pipelineModel.transform(df)

In [171]:
summary = glr.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))

Coefficient Standard Errors: [0.001093997657943606, 0.33181930455911257, 0.3164896632645171, 0.3453064233586901, 0.4178316374514488, 1.1399509620259067]
T Values: [2.069863467930696, 2.423118661973567, -2.1341705792110597, -3.8812018132536563, -3.713131170203293, -3.5001322041435223]
P Values: [0.038465131844473976, 0.015387899399717408, 0.032828820088079746, 0.00010394154054749194, 0.00020471071816796638, 0.00046502746691134966]
Dispersion: 1.0
Null Deviance: 499.9765175549154
Residual Degree Of Freedom Null: 399
Deviance: 458.51749247589913
Residual Degree Of Freedom: 394
AIC: 470.51749247589913


In [207]:
spark.stop()